# License

    Early Graft Loss and its impact, a kidney transplant analysis. - Part 1: Data Preprocessing.
    Copyright (C) 2020  J.R.M. Martens

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <https://www.gnu.org/licenses/>.

# Setup

###### Global Imports

In [1]:
import pandas as pd
import numpy as np

###### Global Functions

In [2]:
# pandas profiling 
import pandas_profiling as pp    # create pandas profiling report
import os.path                   # to check whether file exists
import webbrowser                # open pp reports

def profile(data, file='temp'):
    # Drop id column (unique)
    data = data.drop(columns=['id'])
    
    # Define output filename
    folder = '../EDA/pp/'
    extension = '.html'
    filename = folder + file + extension
    while os.path.isfile(filename):
        file += '#'
        filename = folder + file + extension
        
    # Create report and open in browser
    profile = pp.ProfileReport(data)
    # rejected_variables = profile.get_rejected_variables(threshold=0.9)
    profile.to_file(outputfile=filename)
    webbrowser.open('file://' + os.path.realpath(filename))

In [3]:
### Categorize text columns into numeric using a mapping
def categorize(df, column, values, other = -1, drop = True, nan = False, name = None, replace = False):
    
    # Do nothing if column does not exist
    if column not in df.columns:
        print("Column does not exist (anymore).")
        return df
    
    # Print report 1/2
    print("Mapping:\t", values)
    print("Other value:\t", other)
    print()
    print(column, "(original):\n", df[column].unique())
    
    # Set new column name
    if name != None:
        new_column = name
    elif not replace: 
            new_column = column + '_cat'
    else: 
        new_column = column
        drop = False
    
    # Create new categorized column with mapped values and fill unmapped values to 'other' valu
    if nan: values[np.nan] = -2
    df[new_column] = df[column].map(values).fillna(other).astype(int)
    if nan: df[new_column] = df[new_column].replace({-2: np.nan})
    
    # Print report 2/2
    print("-->")
    print(new_column, "(new):\n", sorted(df[new_column].unique()))
    print()
    
    # Drop old column
    if drop: df = df.drop(columns=[column])
        
    return df

In [4]:
### Sort mappings on values
import operator

def sort_mapping(to_sort):
    sorted_x = sorted(to_sort.items(), key=operator.itemgetter(1))
    for x in sorted_x:
        print('    "' + str(x[0]) + '": ' + str(x[1]) + ',')

In [5]:
### Print length difference of dataframe
def printlen(prev_length):
    new_length = len(df)
    difference = prev_length - new_length
    print("Removed:", difference)
    print("Length:\t", new_length)
    print()
    return new_length

In [6]:
### Remove a certain value from columns
def remove_value(columns, value):
    for column in columns:
        amount = len(df[df[column] == value])
        df[column] = df[column].replace(value, np.nan)
        print("Removed value", value, amount, "times from column", column)

In [7]:
### Set a minimum and/or maximum value for a column
def cutoff(columns, minimum = None, maximum = None):
    for column in columns:
        print(column)
        if minimum != None: 
            old_min = df[column].min()
            amt_below = len(df[df[column] < minimum])
            df[column] = df[column].mask(df[column] < minimum)
        if maximum != None: 
            old_max = df[column].max()
            amt_above = len(df[df[column] > maximum])
            df[column] = df[column].mask(df[column] > maximum)

        if minimum != None: 
            print("Old Min:\t", old_min)
            print("Amount below:\t", amt_below)
            print("New Min:\t", df[column].min())
            print()
        if maximum != None: 
            print("Old Max:\t", old_max)
            print("Amount above:\t", amt_above)
            print("New Max:\t", df[column].max())
            print()

In [8]:
def print_columns():
    for i in sorted(list(df.columns)):
        print(i)

# Data Loading

In [9]:
%%time
# Set date values
dates = ['Date_of_admission', 'Donor_date_of_death', 'Date_of_first_dialysis', 'Nephrectomy_at', 'Date_of_transplant', 'Initial_Graft_fail_date', 'Recipient_date_of_death', 'Initial_Last_seen_date', 'Date_seen__max_', 'Diabetes _since', 'Hypertension _since']
# Load in data 
file = '../data/data_original.csv'
df = pd.read_csv(file, sep=';', decimal=',', na_values=' ', parse_dates=dates)
# Create id column
df['id'] = df.index

CPU times: user 2.72 s, sys: 15.8 ms, total: 2.74 s
Wall time: 2.74 s


# Data removal

In [10]:
remove = [
    'Diuresis__during__last_n_hours', # Not relevant, number of mililiters peed. Probably underreported and clearing is used. Creatinine more important.
    'Diuresis_n_hours', # See above.
    'Urine_glucose', # Not important, creatinine more important.
    'Urine_protein', # Not important, creatinine more important.
    'Multi_organ_donor', # Not important, no impact on recipient.
    'Cold_perfusion_aorta_started_at', # Ischaemi time is decided based on this and used for analysis.
    'Initial_On_machine_indicator', # Too many missing values.
    'Transplant_Kidney_order_nr_organ', # Numbers that are not important to us (1-6).
    'Organ_specification', # Not needed, information is in r_combined_transplants
    'Initial_Graft_fail_spec', # Is 'Other' in Initial_Graft_fail_cause
    'Recipient_death_comment', # Is 'Other' in Recipient_cause_of_death
    'Highest_creatinine_µmol_l', # Use last
]

df = df.drop(columns=remove)

# Data renaming

In [11]:
original_names = df.columns

renames = {
    'fictief_donornummer': 'd_id',
    'fictief_recipientnummer': 'r_id',
    'Type_cadaveric': 'd_cadaveric_type',
    'NHB_Category': 'd_NHB',
    'Donor_cause_of_death': 'd_death_cause',
    'Donor_age': 'd_age',
    'Donor_sex': 'd_sex',
    'Donor_height': 'd_height', 
    'Donor_weight': 'd_weight',
    'Hypertension': 'd_hypertension',
    'Hypertension _since': 'd_hypertension_since',
    'Diabetes': 'd_diabetes',
    'Diabetes _since': 'd_diabetes_since',
    'Smoking': 'd_smoking',
    'Smoking_–_pack_years': 'd_smoking_pack_years',
    'Date_of_admission': 'd_admission_date',
    'Donor_date_of_death': 'd_death_date',
    'Cardiac_arrest': 'd_cardiac_arrest',
    'Hypotensive_periods': 'd_hypotensive_periods',
    'Total_duration_of_hypotensive_episodes': 'd_hypotensive_periods_duration',
#     'Highest_creatinine_µmol_l': 'd_creatinine_highest', 
    'Last_creatinine_µmol_l': 'd_creatinine',
    'Preservation_solution_type': 'preservation_solution_type',
    'Preservation_solution_volume__ml_': 'preservation_solution_volume', # Unit Symbol: in ML
    'Recipient_age': 'r_age',
    'Recipient_sex': 'r_sex',
    'Initial_height': 'r_height',
    'Initial_weight': 'r_weight',
    'Initial_Primary_Disease': 'r_primary_disease',
    'Recipient_blood_group': 'r_blood_group',
    'Transplant___PRA': 'r_PRA',
    'Date_of_first_dialysis': 'r_first_dialysis_date',
    'Last_dialysis_technique__renine_': 'r_last_dialysis_technique',
    'Warm_ischaemic_period_1': 'ischaemic_period_warm_1',
    'Nephrectomy_at': 'd_nephrectomy_date',
    'Initial_Warm_ischaemic_period_2': 'ischaemic_period_warm_2',
    'Initial_Cold_ischaemic_period': 'ischaemic_period_cold',
    'Date_of_transplant': 'r_transplant_date',
    'Mismatch_DR': 'mismatch_DR',
    'Mismatch_A': 'mismatch_A',
    'Mismatch_B': 'mismatch_B',
    'Combined_transplants': 'r_combined_transplants',
    'Initial_Pre_emptive_transplant': 'r_pre_emptive_transplant',
    'Delayed_graft_function': 'r_delayed_graft_function',
    'M3_creatinine': 'r_creatinine_M3',
    'M3_proteinuria': 'r_proteinuria_M3',
    'Y01_creatinine': 'r_creatinine_Y1',
    'Y01_proteinuria': 'r_proteinuria_Y1',
    'Y05_creatinine': 'r_creatinine_Y5',
    'Y05_proteinuria': 'r_proteinuria_Y5',
    'Initial_Graft_fail_date': 'r_graft_fail_date',
    'Initial_Graft_fail_cause': 'r_graft_fail_cause',
    'Recipient_date_of_death': 'r_death_date',
    'Recipient_cause_of_death': 'r_death_cause',
    'Initial_Last_seen_date': 'r_seen_last_date',
    'Date_seen__max_': 'r_seen_max_date',
}

df.rename(columns=renames, inplace=True)

new_names = df.columns

# Categorize

In [12]:
# d_cadaveric_type
column = 'd_cadaveric_type'
mapping = {
    'Non-heartbeating': 0, 
    'Heartbeating': 1
}
df = categorize(df, column, mapping, replace=True)

Mapping:	 {'Non-heartbeating': 0, 'Heartbeating': 1}
Other value:	 -1

d_cadaveric_type (original):
 ['Heartbeating' 'Non-heartbeating']
-->
d_cadaveric_type (new):
 [0, 1]



In [13]:
# d_cardiac_arrest
column = 'd_cardiac_arrest'
mapping = {
    'No': 0, 
    'Yes': 1
}
df = categorize(df, column, mapping, nan=True, replace=True)

Mapping:	 {'No': 0, 'Yes': 1}
Other value:	 -1

d_cardiac_arrest (original):
 ['No' 'Yes' nan]
-->
d_cardiac_arrest (new):
 [0.0, 1.0, nan]



In [14]:
# d_hypotensive_periods
column = 'd_hypotensive_periods'
mapping = {
    'N': 0, 
    'Y': 1
}
df = categorize(df, column, mapping, nan=True, replace=True)

Mapping:	 {'N': 0, 'Y': 1}
Other value:	 -1

d_hypotensive_periods (original):
 ['N' 'Y' nan]
-->
d_hypotensive_periods (new):
 [0.0, 1.0, nan]



In [15]:
# r_delayed_graft_function
column = 'r_delayed_graft_function'
mapping = {
    'Never': 0, 
    'Delayed': 1,
    'Direct': 2,
    'Unknown': -2 # Will become nan
}
df = categorize(df, column, mapping, nan=True, replace=True)

Mapping:	 {'Never': 0, 'Delayed': 1, 'Direct': 2, 'Unknown': -2}
Other value:	 -1

r_delayed_graft_function (original):
 ['Direct' 'Unknown' nan 'Delayed' 'Never']
-->
r_delayed_graft_function (new):
 [2.0, nan, 0.0, 1.0]



In [16]:
# diabetes
column = 'd_diabetes'
mapping = {
    '1': 1,
    '2': 2,
    'Y': 3, 
    'N': 0,
    'U': -2 # Will become nan
}
df = categorize(df, column, mapping, nan=True, replace=True)

Mapping:	 {'1': 1, '2': 2, 'Y': 3, 'N': 0, 'U': -2}
Other value:	 -1

d_diabetes (original):
 [nan 'N' '2' 'U' 'Y' '1']
-->
d_diabetes (new):
 [nan, 0.0, 1.0, 2.0, 3.0]



In [17]:
# hypertension
column = 'd_hypertension'
mapping = {
    'N': 0,
    'Y': 1, 
    'U': -2 # Will become nan
}
df = categorize(df, column, mapping, nan=True, replace=True)

Mapping:	 {'N': 0, 'Y': 1, 'U': -2}
Other value:	 -1

d_hypertension (original):
 [nan 'Y' 'N' 'U']
-->
d_hypertension (new):
 [nan, 0.0, 1.0]



In [18]:
# smoking
column = 'd_smoking'
mapping = {
    'N': 0,
    'Y': 1,
    'U': -2 # Will become nan
}
df = categorize(df, column, mapping, nan=True, replace=True)

Mapping:	 {'N': 0, 'Y': 1, 'U': -2}
Other value:	 -1

d_smoking (original):
 [nan 'Y' 'N' 'U']
-->
d_smoking (new):
 [nan, 0.0, 1.0]



In [19]:
# d_sex
column = 'd_sex'
mapping = {
    'Male': 0, 
    'Female': 1
}
df = categorize(df, column, mapping, replace=True)

# r_sex
column = 'r_sex'
mapping = {
    'Male': 0, 
    'Female': 1
}
df = categorize(df, column, mapping, replace=True)

Mapping:	 {'Male': 0, 'Female': 1}
Other value:	 -1

d_sex (original):
 ['Female' 'Male']
-->
d_sex (new):
 [0, 1]

Mapping:	 {'Male': 0, 'Female': 1}
Other value:	 -1

r_sex (original):
 ['Male' 'Female']
-->
r_sex (new):
 [0, 1]



In [20]:
# r_combined_transplants
column = 'r_combined_transplants'
mapping = {
    'LKi': 0, 
    'RKi': 1, 
    'LKi RKi': 2
}
other = 3
df = categorize(df, column, mapping, other=other, replace=True)

Mapping:	 {'LKi': 0, 'RKi': 1, 'LKi RKi': 2}
Other value:	 3

r_combined_transplants (original):
 ['RKi' 'LKi' 'LKi RKi' 'Pa RKi' 'LKi Pa' 'RKi WLiv' 'LKi WLiv'
 'ERLLiv LKi']
-->
r_combined_transplants (new):
 [0, 1, 2, 3]



In [21]:
# d_NHB --> d_NHB_cat
column = 'd_NHB'
mapping = {
    'I: brought in dead (i.e. in the hospital.)': 1,
    'II: unsuccessful resuscitation.': 2,
    'III: awaiting cardiac arrest (i.e. treatment stopped e.g. ventilator switch off.)': 3,
    'IV: cardiac arrest after brain stem death.': 4,
}
df = categorize(df, column, mapping, nan=True)

Mapping:	 {'I: brought in dead (i.e. in the hospital.)': 1, 'II: unsuccessful resuscitation.': 2, 'III: awaiting cardiac arrest (i.e. treatment stopped e.g. ventilator switch off.)': 3, 'IV: cardiac arrest after brain stem death.': 4}
Other value:	 -1

d_NHB (original):
 [nan
 'III: awaiting cardiac arrest (i.e. treatment stopped e.g. ventilator switch off.)'
 'IV: cardiac arrest after brain stem death.'
 'II: unsuccessful resuscitation.'
 'I: brought in dead (i.e. in the hospital.)']
-->
d_NHB_cat (new):
 [nan, 1.0, 2.0, 3.0, 4.0]



In [22]:
# r_graft_fail_cause
column = 'r_graft_fail_cause'
mapping = {
    np.nan: 0,
    'Patient died with functioning transplant': 1,
    'Rejection after stopping all immunosuppressive drugs': 2,
    'Rejection while taking immunosuppressive drugs (acute / chronic)': 3,
    'Hyperacute Rejection': 4,
    'Non-Viable Kidney': 5,
    'Permanent Non-Function': 6,
    'Recurrent primary renal disease': 7,
    'Infection ( not graft related )': 8,
    'Infection of graft': 9,
    'Thrombosis / Infarction': 10,
    'Technical problems': 11,
    'Vascular or Ureteric operative problems': 12,
    'Vascular problems: not operative or rejection related': 13,
    'Removal of functioning graft': 14,
    'Other ( renal )': 15,
    'Unkown': 16
}
df = categorize(df, column, mapping)

Mapping:	 {nan: 0, 'Patient died with functioning transplant': 1, 'Rejection after stopping all immunosuppressive drugs': 2, 'Rejection while taking immunosuppressive drugs (acute / chronic)': 3, 'Hyperacute Rejection': 4, 'Non-Viable Kidney': 5, 'Permanent Non-Function': 6, 'Recurrent primary renal disease': 7, 'Infection ( not graft related )': 8, 'Infection of graft': 9, 'Thrombosis / Infarction': 10, 'Technical problems': 11, 'Vascular or Ureteric operative problems': 12, 'Vascular problems: not operative or rejection related': 13, 'Removal of functioning graft': 14, 'Other ( renal )': 15, 'Unkown': 16}
Other value:	 -1

r_graft_fail_cause (original):
 ['Recurrent primary renal disease'
 'Rejection while taking immunosuppressive drugs (acute / chronic)' nan
 'Patient died with functioning transplant' 'Technical problems'
 'Other ( renal )' 'Permanent Non-Function' 'Infection of graft'
 'Thrombosis / Infarction' 'Removal of functioning graft'
 'Vascular problems: not operative or re

In [23]:
# preservation_solution_type
column = 'preservation_solution_type'
mapping = {
    'Bretschneider': 0,
    'HTK / Bretschneider': 1,
    'Celsior': 2,
    'Eurocollins': 3,
    'IGL-1': 4,
    'Hartmann': 5,
    'UW': 6,
    'Modified UW': 7,
    'Other': 8,
    np.nan: 9
}
df = categorize(df, column, mapping)

Mapping:	 {'Bretschneider': 0, 'HTK / Bretschneider': 1, 'Celsior': 2, 'Eurocollins': 3, 'IGL-1': 4, 'Hartmann': 5, 'UW': 6, 'Modified UW': 7, 'Other': 8, nan: 9}
Other value:	 -1

preservation_solution_type (original):
 ['UW' 'Eurocollins' 'HTK / Bretschneider' nan 'Other' 'Bretschneider'
 'Hartmann' 'Modified UW' 'Celsior' 'IGL-1']
-->
preservation_solution_type_cat (new):
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]



In [24]:
# r_last_dialysis_technique
column = 'r_last_dialysis_technique'
mapping = {
    'Actieve centrumhaemodialyse': 0,
    'CAPD': 1,
    'CCPD/APD': 2,
    'Centrumhaemodialyse, vorm onbekend': 3,
    'Dialyse, vorm onbekend': 4,
    'Geen voorafgaande dialyse': 5,
    'IPD': 6,
    'Nachtelijke centrum-HD': 7,
    'Passieve centrumhaemodialyse': 8,
    'Thuishaemodialyse': 9,
    'Transplantaat-falen': 10,
    'Onbekend': 11,
    np.nan: 11
}
df = categorize(df, column, mapping)

Mapping:	 {'Actieve centrumhaemodialyse': 0, 'CAPD': 1, 'CCPD/APD': 2, 'Centrumhaemodialyse, vorm onbekend': 3, 'Dialyse, vorm onbekend': 4, 'Geen voorafgaande dialyse': 5, 'IPD': 6, 'Nachtelijke centrum-HD': 7, 'Passieve centrumhaemodialyse': 8, 'Thuishaemodialyse': 9, 'Transplantaat-falen': 10, 'Onbekend': 11, nan: 11}
Other value:	 -1

r_last_dialysis_technique (original):
 ['Centrumhaemodialyse, vorm onbekend' 'CAPD' 'Geen voorafgaande dialyse'
 'Thuishaemodialyse' 'Transplantaat-falen' 'CCPD/APD' nan
 'Passieve centrumhaemodialyse' 'IPD' 'Actieve centrumhaemodialyse'
 'Onbekend' 'Dialyse, vorm onbekend' 'Nachtelijke centrum-HD']
-->
r_last_dialysis_technique_cat (new):
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]



In [25]:
# r_pre_emptive_transplant
column = 'r_pre_emptive_transplant'
mapping = {
    'No': 0,
    'Yes': 1,
    'Unknown': -2 # Will become nan
}
df = categorize(df, column, mapping, nan=True, replace=True)

Mapping:	 {'No': 0, 'Yes': 1, 'Unknown': -2}
Other value:	 -1

r_pre_emptive_transplant (original):
 ['No' nan 'Yes' 'Unknown']
-->
r_pre_emptive_transplant (new):
 [0.0, nan, 1.0]



In [26]:
# d_death_cause - text to 53 categories, 53 to 4 categories (missing)
# 1 = Trauma, 2 = Stroke, 3 = Cardiac arrest, 4 = Other
column = 'd_death_cause'
mapping = {
    "Trauma: Capitis": 1,
    "Suicide: Head injury": 1,
    "Trauma: Not Otherwise Specified": 1,
    "Trauma: Travel accident on land": 1,
    "SDH: Sub Dural Hematoma": 1,
    "Suicide: Jump": 1,
    "Trauma: Falling": 1,
    "Trauma: Suffocation": 1,
    "Trauma: Exposure to gasses / smoke / vapour": 1,
    "Trauma: Drowning": 1,
    "Trauma: Travel accident on water": 1,
    "Trauma: Mechanical": 1,
    "EDH: Epi Dural Hematoma": 1,
    "CVA: Cerebro Vascular Accident Not Otherwise Specified": 2,
    "CVA: Intra Cerebral Bleeding": 2,
    "CVA: Cerebral Ischemia": 2,
    "SAB: Sub Arachnoidal Bleeding": 2,
    "Circulational: Acute Myocard Infarct": 3,
    "Circulational: Cardiac Arrest": 3,
    "Circulational: Recidief Myocard Infarct": 3,
    "Non-accident: Not Otherwise Specified": 4,
    "Circulational: Not Otherwise Specified": 4,
    "Brain Tumor: Not Otherwise Specified": 4,
    "Respirational: Not Otherwise Specified": 4,
    "Not Otherwise Specified": 4,
    "Suicide: Not Otherwise Specified": 4,
    "Malignancies": 4,
    "Suicide: Drugs": 4,
    "Brain Tumor: Astrocytoma grade 1 or 2": 4,
    "Respirational: Status Asthmaticus": 4,
    "Brain Tumor: Malignant": 4,
    "Suicide: Respiratory": 4,
    "Brain Tumor: Benign": 4,
    "Medical complication: Medication": 4,
    "Status Epilepticus": 4,
    "Meningitis: Bacterial": 4,
    np.nan: 4,
    "Medical complication: Surgical / Medical treatment": 4,
    "Medical complication: Diagnostic treatment": 4,
    "Meningitis: Not Otherwise Specified": 4,
    "Respirational: Epiglotitis / Laryngitis": 4,
    "Brain Tumor: Astrocytoma grade 3": 4,
}
df = categorize(df, column, mapping)

Mapping:	 {'Trauma: Capitis': 1, 'Suicide: Head injury': 1, 'Trauma: Not Otherwise Specified': 1, 'Trauma: Travel accident on land': 1, 'SDH: Sub Dural Hematoma': 1, 'Suicide: Jump': 1, 'Trauma: Falling': 1, 'Trauma: Suffocation': 1, 'Trauma: Exposure to gasses / smoke / vapour': 1, 'Trauma: Drowning': 1, 'Trauma: Travel accident on water': 1, 'Trauma: Mechanical': 1, 'EDH: Epi Dural Hematoma': 1, 'CVA: Cerebro Vascular Accident Not Otherwise Specified': 2, 'CVA: Intra Cerebral Bleeding': 2, 'CVA: Cerebral Ischemia': 2, 'SAB: Sub Arachnoidal Bleeding': 2, 'Circulational: Acute Myocard Infarct': 3, 'Circulational: Cardiac Arrest': 3, 'Circulational: Recidief Myocard Infarct': 3, 'Non-accident: Not Otherwise Specified': 4, 'Circulational: Not Otherwise Specified': 4, 'Brain Tumor: Not Otherwise Specified': 4, 'Respirational: Not Otherwise Specified': 4, 'Not Otherwise Specified': 4, 'Suicide: Not Otherwise Specified': 4, 'Malignancies': 4, 'Suicide: Drugs': 4, 'Brain Tumor: Astrocytoma g

In [27]:
# r_death_cause - text to 53 categories, 53 to 7 categories
# 0 = Unknown, 1 = Other, 2 = Pulmonary, 3 = Cardiovascular, 4 = Cerebrovascular, 5 = GI - Liver, 6 = Renal / dialysis related
column = 'r_death_cause'
mapping = {
    'Cause of death: uncertain - not determined': 0,
    'Unknown': 0,
    'Septicemia / MOF': 1,
    'Patient refused further dialysis treatment': 1,
    'Malignant disease - Other / Specify': 1,
    'Malignant disease - Lymfoproliferative disorder / Specify': 1,
    'Other / Specify': 1,
    'Infection': 1,
    'Cachexia': 1,
    'Suicide': 1,
    'Tuberculosis ( not Lung )': 1,
    'Surgical complications': 1,
    'Hemorrhage / Other / Specify': 1,
    'Dialysis therapy ceased for any other reason': 1,
    'Infections elsewhere (except Viral Hepatitis)': 1,
    'Accident / All causes / Specify': 1,
    'Bone Marrow Depression': 1,
    'Viral infection / generalized': 1,
    'Hemorrhage from surgery': 1,
    'Dementia': 1,
    'Liver disease due to other Viral Hepatitis': 1,
    'Accident unrelated to dialysis treatment': 1,
    'Pulmonary Embolus': 2,
    'Pulmonary infection ( Bacterial )': 2,
    'Pulmonary infection ( Viral )': 2,
    'Pulmonary infection ( Fungal or Protozoal )': 2,
    'Tuberculosis ( Lung )': 2,
    'Causes of cardiac failure / other': 3,
    'Myocardial ischemia / infarction': 3,
    'Cardiac arrest - Sudden death': 3,
    'Fluid Overload': 3,
    'Hypertensive Cardiac Failure': 3,
    'Hemorrhage from ruptured vascular aneurysm (AAA / etc)': 3,
    'Cerebrovascular accident (CVA)': 4,
    'Pancreatitis': 5,
    'Mesenteric Infarction': 5,
    'Gastro-intestinal / Other / Specify': 5,
    'Cirrhosis - not viral': 5,
    'Gastro-intestinal hemorrhage': 5,
    'Peritonitis': 5,
    'Cystic Liver Disease': 5,
    'Liver failure / Cause Unknown': 5,
    'Perforation of colon': 5,
    'Perforation of peptic ulcer': 5,
    'Liver disease due to Hepatitis B Virus': 5,
    'Liver disease due to Drug Toxicity': 5,
    'Hyperkalemia': 6,
    'Accident related to dialysis treatment': 6,
    'Failure of transplant': 6,
    'Sclerosing peritoneal disease': 6,
    'Renal failure': 6,
    'Hemorrhage from graft site': 6,
    'Hemorrhage from vascular access or dialysis circuit': 6,
}
df = categorize(df, column, mapping, nan=True)

Mapping:	 {'Cause of death: uncertain - not determined': 0, 'Unknown': 0, 'Septicemia / MOF': 1, 'Patient refused further dialysis treatment': 1, 'Malignant disease - Other / Specify': 1, 'Malignant disease - Lymfoproliferative disorder / Specify': 1, 'Other / Specify': 1, 'Infection': 1, 'Cachexia': 1, 'Suicide': 1, 'Tuberculosis ( not Lung )': 1, 'Surgical complications': 1, 'Hemorrhage / Other / Specify': 1, 'Dialysis therapy ceased for any other reason': 1, 'Infections elsewhere (except Viral Hepatitis)': 1, 'Accident / All causes / Specify': 1, 'Bone Marrow Depression': 1, 'Viral infection / generalized': 1, 'Hemorrhage from surgery': 1, 'Dementia': 1, 'Liver disease due to other Viral Hepatitis': 1, 'Accident unrelated to dialysis treatment': 1, 'Pulmonary Embolus': 2, 'Pulmonary infection ( Bacterial )': 2, 'Pulmonary infection ( Viral )': 2, 'Pulmonary infection ( Fungal or Protozoal )': 2, 'Tuberculosis ( Lung )': 2, 'Causes of cardiac failure / other': 3, 'Myocardial ischemia

In [28]:
# r_primary_disease
column = 'r_primary_disease'
mapping = {
    "Pyelonephritis/Interstitial nephritis - VU reflux without obstruction": 0,
    "Polycystic Kidneys - Adult type (dominant)": 0,
    "Kidney tumor": 0,
    "Hereditary/Familial nephropathy - Type unspecified": 0,
    "Congenital renal hypoplasia - Type unspecified": 0,
    "Nephropathy due to analgesic drugs": 0,
    "Hereditary nephritis with nerve deafness (Alport's syndrome)": 0,
    "Medullary Cystic Disease - including nephronophthisis": 0,
    "Pyelonephritis/Interstitial nephritis - Congenital obstructive uropathy": 0,
    "Cystic kidney disease - Type unspecified": 0,
    "Congenital renal dysplasia with/without urinary tract malformation": 0,
    "Syndrome of agenesis of abdominal muscles (Prune Belly syndrome)": 0,
    "Tuberculosis": 0,
    "Hereditary Nephropathy - Other": 0,
    "Pyelonephritis/Interstitial nephritis - Acquired obstructive uropathy": 0,
    "Cystinosis": 0,
    "Rapidly progressive Glomerulonephritis without systemic disease": 0,
    "Traumatic or Surgical loss of kidney": 0,
    "Polycystic Kidneys - Infantile type (recessive)": 0,
    "Nephropathy caused by other specific drug": 0,
    "Nephropathy due to Cyclosporin A": 0,
    "Cystic kidney disease - Other specified type": 0,
    "Nephropathy due to Cisplatinum": 0,
    "Iscemic renal disease / cholesterol embolism": 0,
    "Focal segmental glomerulosclerosis with nephrotic syndrome in adults": 1,
    "Lupus Erythematosus": 1,
    "Focal segmental glomerulosclerosis with nephrotic syndrome in children": 1,
    "Glomerulonephritis - histologically examined": 1,
    "Glomerulonephritis - histologically not examined": 1,
    "Diabetes Type I": 1,
    "Hemolytic Uremic Syndrome including Moschcowitz syndrome": 1,
    "Renal vascular disease due to malignant hypertension": 1,
    "Renal vascular disease due to hypertension": 1,
    "Nephrocalcinosis and Hypercalcemic Nephropathy": 1,
    "Wegener's granulomatosis": 1,
    "Amyloidosis": 1,
    "Goodpasture's syndrome": 1,
    "Henoch-Schonlein Purpura": 1,
    "IgA nephropathy (proven by immunofluorescence)": 1,
    "Primary oxalosis": 1,
    "Membranous nephropathy": 1,
    "Gout nephropathy (urate)": 1,
    "Dense deposit disease MPGN - Type II": 1,
    "Pyelonephritis/Interstitial nephritis - Neurogenic bladder": 1,
    "Renal vascular disease due to polyarteritis": 1,
    "Myelomatosis - Light chain deposit disease": 1,
    "Membrano-proliferative glomerulonephritis - Type I": 1,
    "Systemic sclerosis (scleroderma)": 1,
    "Diabetes Type II": 1,
    "Fabry's disease": 1,
    "Cryoglobulinemic glomerulonephritis": 1,
    "Chronic renal failure - etiology uncertain": 2,
    "Pyelonephritis/Interstitial nephritis - Urolithiasis": 2,
    "Tubulo-interstitial nephritis (not Pyelonephritis)": 2,
    "Pyelonephritis/Interstitial nephritis - Cause not specified": 2,
    "Renal vascular disease - Type unspecified": 2,
    "Other identified renal disorders - Specify": 2,
    "Pyelonephritis/Interstitial nephritis - Other cause": 2,
    "Renal vascular disease - Classified": 2,
    "Cortical or Tubular necrosis": 2,
    "Multisystem disease - Other specified type": 2,
}
df = categorize(df, column, mapping, nan=True, name='r_disease_recurrent')

Mapping:	 {'Pyelonephritis/Interstitial nephritis - VU reflux without obstruction': 0, 'Polycystic Kidneys - Adult type (dominant)': 0, 'Kidney tumor': 0, 'Hereditary/Familial nephropathy - Type unspecified': 0, 'Congenital renal hypoplasia - Type unspecified': 0, 'Nephropathy due to analgesic drugs': 0, "Hereditary nephritis with nerve deafness (Alport's syndrome)": 0, 'Medullary Cystic Disease - including nephronophthisis': 0, 'Pyelonephritis/Interstitial nephritis - Congenital obstructive uropathy': 0, 'Cystic kidney disease - Type unspecified': 0, 'Congenital renal dysplasia with/without urinary tract malformation': 0, 'Syndrome of agenesis of abdominal muscles (Prune Belly syndrome)': 0, 'Tuberculosis': 0, 'Hereditary Nephropathy - Other': 0, 'Pyelonephritis/Interstitial nephritis - Acquired obstructive uropathy': 0, 'Cystinosis': 0, 'Rapidly progressive Glomerulonephritis without systemic disease': 0, 'Traumatic or Surgical loss of kidney': 0, 'Polycystic Kidneys - Infantile type

# Delete

In [29]:
length = len(df)
original = length

In [30]:
years = list(range(1990, 2018))
# Create selectable for each year
ix_year = dict()
for year in years:
    start = str(year) + '-01-01'
    end = str(year) + '-12-31'
    ix_year[year] = ((df['r_transplant_date'] >= start) & (df['r_transplant_date'] <= end))
    
for year in range(2013, 2018):
    print(year, len(df[ix_year[year]]))

2013 436
2014 475
2015 468
2016 390
2017 304


In [31]:
# Print original length
print("Original:", original)
print()

Original: 11416



In [32]:
# Remove duplicates (always both kidneys)
df = df[df.duplicated(['r_id','r_transplant_date']) == False]
length = printlen(length)

Removed: 58
Length:	 11358



In [33]:
# Remove recipients aged < 12 years
df = df.loc[df['r_age'] >= 12]
length = printlen(length)

Removed: 253
Length:	 11105



In [34]:
# Remove other from combined transplants
df = df[df['r_combined_transplants'] != 3]
length = printlen(length)

Removed: 533
Length:	 10572



In [35]:
years = list(range(1990, 2018))
# Create selectable for each year
ix_year = dict()
for year in years:
    start = str(year) + '-01-01'
    end = str(year) + '-12-31'
    ix_year[year] = ((df['r_transplant_date'] >= start) & (df['r_transplant_date'] <= end))
    
for year in range(2013, 2018):
    print(year, len(df[ix_year[year]]))

2013 408
2014 436
2015 435
2016 370
2017 286


In [36]:
# Remove NHB Category I (brought in dead) and II (unsuccesful resuscitation)
df = df[df['d_NHB_cat'] != 1]
df = df[df['d_NHB_cat'] != 2] 
length = printlen(length)

# Print final length
final = len(df)
difference = original - final
print("Original:\t", original, sep='')
print("Removed:\t", difference)
print("Final:\t\t", final)

Removed: 212
Length:	 10360

Original:	11416
Removed:	 1056
Final:		 10360


## Date Differences

In [37]:
# df_dates = df.select_dtypes(include=['datetime64']).columns # Select all date columns

# All dates except for r_transplant_date
dates_diff = {
    'd_admission_date': 'd_admission_date_diff',
    'd_death_date': 'd_death_date_diff',
    'r_first_dialysis_date': 'r_first_dialysis_date_diff',
    'd_nephrectomy_date': 'd_nephrectomy_date_diff',
    'r_graft_fail_date': 'r_graft_fail_date_diff',
    'r_death_date': 'r_death_date_diff',
    'r_seen_last_date': 'r_seen_last_date_diff',
    'r_seen_max_date': 'r_seen_max_date_diff',
}

In [38]:
for date in dates_diff.keys():
    df[dates_diff[date]] = ((df[date] - df['r_transplant_date']) / np.timedelta64(1, 'D'))
#     df[dates_diff[date]] = (df[date] - df['r_transplant_date']).dt.days
    df = df.drop(columns=[date])

## Data Validation

### -1 and 0 removal

In [39]:
# Remove -1 values
value = -1

columns = ['r_weight', 'r_height', 'ischaemic_period_warm_2', 'r_proteinuria_M3', 'r_proteinuria_Y1', 'r_proteinuria_Y5']
remove_value(columns, value)

Removed value -1 205 times from column r_weight
Removed value -1 204 times from column r_height
Removed value -1 1 times from column ischaemic_period_warm_2
Removed value -1 90 times from column r_proteinuria_M3
Removed value -1 22 times from column r_proteinuria_Y1
Removed value -1 65 times from column r_proteinuria_Y5


In [40]:
# Remove 0 values
value = 0

columns = ['d_height', 'd_weight']
remove_value(columns, value)

Removed value 0 76 times from column d_height
Removed value 0 70 times from column d_weight


## Cutoff

In [41]:
# Set minimum and maximum for creatinine values
columns = ['r_creatinine_M3', 'r_creatinine_Y1', 'r_creatinine_Y5', 'd_creatinine'] # ', 'd_creatinine_highest'
minimum = 30
maximum = 1000
cutoff(columns, minimum = minimum, maximum = maximum)

r_creatinine_M3
Old Min:	 -1.0
Amount below:	 43
New Min:	 31.0

Old Max:	 2373.0
Amount above:	 7
New Max:	 973.0

r_creatinine_Y1
Old Min:	 -1.0
Amount below:	 2
New Min:	 41.0

Old Max:	 1377.0
Amount above:	 4
New Max:	 860.0

r_creatinine_Y5
Old Min:	 -1.0
Amount below:	 6
New Min:	 45.0

Old Max:	 1318.0
Amount above:	 1
New Max:	 999.0

d_creatinine
Old Min:	 0.09
Amount below:	 155
New Min:	 30.0

Old Max:	 13260.0
Amount above:	 44
New Max:	 972.4



In [42]:
# Set minimum and maximum for d_nephrectomy_date_diff
columns = ['d_admission_date_diff']
minimum = -180
cutoff(columns, minimum = minimum)

d_admission_date_diff
Old Min:	 -3291.0
Amount below:	 14
New Min:	 -128.0



In [43]:
# Set minimum and maximum for d_nephrectomy_date_diff
columns = ['d_nephrectomy_date_diff']
minimum = -28
maximum = 0
cutoff(columns, minimum = minimum, maximum=maximum)

d_nephrectomy_date_diff
Old Min:	 -29417.0
Amount below:	 55
New Min:	 -14.0

Old Max:	 2739.0
Amount above:	 33
New Max:	 0.0



In [44]:
# Set minimum for date values that should occur after r_transplant_date
columns = ['r_seen_last_date_diff']
minimum = 0
cutoff(columns, minimum = minimum)

r_seen_last_date_diff
Old Min:	 -5669.0
Amount below:	 3
New Min:	 0.0



In [45]:
# Set maximum for date values that should occur before r_transplant_date
columns = ['r_first_dialysis_date_diff'] # d_nephrectomy_date_diff
maximum = 0
cutoff(columns, maximum = maximum)

r_first_dialysis_date_diff
Old Max:	 300.0
Amount above:	 2
New Max:	 -7.0



# Recompute

In [46]:
# d_diabetes_since from date to year
df['d_diabetes_since'] = df['d_diabetes_since'].apply(lambda x: x.year)

In [47]:
# d_hypertension_since from date to year
df['d_hypertension_since'] = df['d_hypertension_since'].apply(lambda x: x.year)

In [48]:
# r_graft_fail_cause_cat from 0 to 1 if there is a death date - 129 cases
def graft_fail_cause(x):
    if x['r_graft_fail_cause_cat'] == 0:
        if pd.notnull(x['r_death_date_diff']):
            return 1
        else:
            return 0
    else:
        return x['r_graft_fail_cause_cat']
    
df['r_graft_fail_cause_cat'] = df.apply(graft_fail_cause, axis=1)
# df.groupby('r_graft_fail_cause_cat')['id'].nunique()

# Compute

In [49]:
# Death Recipient - 0: No, 1: Yes
def death(x):
    # Set 1 if recipient has a death date
    if pd.notnull(x['r_death_date_diff']):
        return 1
    # Set 1 if r_graft_fail_cause is equal to 'Patient died with functioning transplant'
    elif x['r_graft_fail_cause_cat'] == 1:
        return 1
    else:
        return 0
    
df['r_dead'] = df.apply(death, axis=1)
df.groupby('r_dead')['id'].nunique()

r_dead
0    5947
1    4413
Name: id, dtype: int64

In [50]:
# BMI
df['d_BMI'] = df['d_weight'] / ((df['d_height'] / 100) * (df['d_height'] / 100))
df['r_BMI'] = df['r_weight'] / ((df['r_height'] / 100) * (df['r_height'] / 100))

# Set maximum for BMI values
columns = ['d_BMI', 'r_BMI']
maximum = 100
cutoff(columns, maximum = maximum)

d_BMI
Old Max:	 388.8888888888889
Amount above:	 2
New Max:	 66.59729448491154

r_BMI
Old Max:	 285.9960552268244
Amount above:	 2
New Max:	 48.44290657439447



In [51]:
# MDRD
df['d_MDRD'] = 186*((df['d_creatinine']/88.4)**(-1.154))*((df['d_age'])**(-0.203))*(1-(df['d_sex']*0.258))

df['r_MDRD_M3'] = 186*((df['r_creatinine_M3']/88.4)**(-1.154))*((df['r_age'])**(-0.203))*(1-(df['r_sex']*0.258))
df['r_MDRD_Y1'] = 186*((df['r_creatinine_Y1']/88.4)**(-1.154))*(((df['r_age']+1))**(-0.203))*(1-(df['r_sex']*0.258))
df['r_MDRD_Y5'] = 186*((df['r_creatinine_Y5']/88.4)**(-1.154))*(((df['r_age']+5))**(-0.203))*(1-(df['r_sex']*0.258))

In [52]:
df['d_MDRD'].max()

inf

In [53]:
# Retransplant
nr_transplants = df.groupby('r_id')['id'].nunique()
def retransplant(x):
    if nr_transplants[x['r_id']] == 1:
        return 0
    else:
        indices = list(df[df['r_id'] == x['r_id']].index.values)
        transplant = indices.index(x['id']) + 1
        return transplant
df['retransplant'] = df.apply(retransplant, axis=1)
df.groupby('retransplant')['id'].nunique()

retransplant
0    8631
1     815
2     815
3      87
4      11
5       1
Name: id, dtype: int64

In [54]:
# Retransplant After - a recipient had another transplant after the current transplant
nr_transplants = df.groupby('r_id')['id'].nunique()
def retransplant_after(x):
    if x['retransplant'] == 0:
        return 0
    elif nr_transplants[x['r_id']] == x['retransplant']:
        return 0
    else:
        return 1
df['retransplant_after'] = df.apply(retransplant_after, axis=1)
df.groupby('retransplant_after')['id'].nunique()

retransplant_after
0    9446
1     914
Name: id, dtype: int64

In [55]:
# Retransplant Days - the number of days the recipient had until the next transplant
def retransplant_days(x):
    if x['retransplant_after'] == 0:
        return np.nan
    else:
        nr = x['retransplant']
        nxt = df[((df['r_id'] == x['r_id']) & (df['retransplant'] == (nr+1)))]['r_transplant_date'].iloc[0]
        days = ((nxt - x['r_transplant_date']) / np.timedelta64(1, 'D'))
        return days
    
df['retransplant_days'] = df.apply(retransplant_days, axis=1)

In [56]:
# Graft loss
def graftloss(x):
    # Set 0 if r_graft_fail_cause is equal to 'Patient died with functioning transplant'
    if x['r_graft_fail_cause_cat'] == 0 or x['r_graft_fail_cause_cat'] == 1:
        return 0
    else:
        return 1
    
df['graftloss'] = df.apply(graftloss, axis=1)
df.groupby('graftloss')['id'].nunique()

graftloss
0    7570
1    2790
Name: id, dtype: int64

In [57]:
# Early Graft Loss - 1 if graftloss and graft fail date <= 90 days
def early_graftloss(x):
    if x['graftloss'] == 1:
        if x['r_graft_fail_date_diff'] <= 90:
            return 1
        else:
            return 0
    else:
        return 0
    
df['early_graftloss'] = df.apply(early_graftloss, axis=1)
df.groupby('early_graftloss')['id'].nunique()

early_graftloss
0    9425
1     935
Name: id, dtype: int64

## Merging

In [58]:
# Last seen date of a recipient, use maximum date of seen_last, seen_max or graft_fail (means there was an update)
def seen(x):
    seen_last = x['r_seen_last_date_diff']
    seen_max = x['r_seen_max_date_diff']
    graft_fail = x['r_graft_fail_date_diff']
    lst = [seen_last, seen_max, graft_fail]
    return np.nanmax(lst)
    
df['r_seen_date_diff'] = df.apply(seen, axis=1)
df = df.drop(columns=['r_seen_last_date_diff', 'r_seen_max_date_diff']) # Drop columns last seen

## Exporting

In [59]:
# Load in data 
version_date = '20-04-11'
folder = '../data/'
file = f"data_{version_date}.csv"
filename = folder + file
df.to_csv(filename, sep=';', decimal=',', na_rep=' ', index=False)
print(file, "succesfully created.")

data_20-04-11.csv succesfully created.


# Sandbox
A place to perform wild code.

In [60]:
print("Final columns:\n")
print_columns()

Final columns:

d_BMI
d_MDRD
d_NHB_cat
d_admission_date_diff
d_age
d_cadaveric_type
d_cardiac_arrest
d_creatinine
d_death_cause_cat
d_death_date_diff
d_diabetes
d_diabetes_since
d_height
d_hypertension
d_hypertension_since
d_hypotensive_periods
d_hypotensive_periods_duration
d_id
d_nephrectomy_date_diff
d_sex
d_smoking
d_smoking_pack_years
d_weight
early_graftloss
graftloss
id
ischaemic_period_cold
ischaemic_period_warm_1
ischaemic_period_warm_2
mismatch_A
mismatch_B
mismatch_DR
preservation_solution_type_cat
preservation_solution_volume
r_BMI
r_MDRD_M3
r_MDRD_Y1
r_MDRD_Y5
r_PRA
r_age
r_blood_group
r_combined_transplants
r_creatinine_M3
r_creatinine_Y1
r_creatinine_Y5
r_dead
r_death_cause_cat
r_death_date_diff
r_delayed_graft_function
r_disease_recurrent
r_first_dialysis_date_diff
r_graft_fail_cause_cat
r_graft_fail_date_diff
r_height
r_id
r_last_dialysis_technique_cat
r_pre_emptive_transplant
r_proteinuria_M3
r_proteinuria_Y1
r_proteinuria_Y5
r_seen_date_diff
r_sex
r_transplant_date
r